In [ ]:
#making the directory 
mkdir ./DemultiplexedSeqs

In [ ]:
#importing the data
qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path Fastq \
  --input-format CasavaOneEightSingleLanePerSampleDirFmt \
  --output-path ./DemultiplexedSeqs/demux-paired-end.qza

In [ ]:
qiime demux summarize \
  --i-data ./DemultiplexedSeqs/demux-paired-end.qza \
  --o-visualization ./DemultiplexedSeqs/demux-paired-end.qzv

In [ ]:
qiime tools view ./DemultiplexedSeqs/demux-paired-end.qzv

In [ ]:
#running cutadapt 
qiime cutadapt trim-paired \
  --i-demultiplexed-sequences ./DemultiplexedSeqs/demux-paired-end.qza \
  --p-front-f GTGYCAGCMGCCGCGGTAA \
  --p-front-r GGACTACNVGGGTWTCTAAT \
  --o-trimmed-sequences ./DemultiplexedSeqs/demux-paired-end-trimmed.qza

In [ ]:
qiime demux summarize \
  --i-data ./DemultiplexedSeqs/demux-paired-end-trimmed.qza  \
  --o-visualization ./DemultiplexedSeqs/demux-paired-end-trimmed.qzv

In [ ]:
mkdir ./DADA2

In [ ]:
#running dada2 
qiime dada2 denoise-paired \
  --i-demultiplexed-seqs ./DemultiplexedSeqs/demux-paired-end-trimmed.qza \
  --p-trim-left-f 0 \
  --p-trunc-len-f 240 \
  --p-trim-left-r 0 \
  --p-trunc-len-r 180 \
  --o-representative-sequences ./DADA2/rep-seqs-dada2.qza \
  --o-table ./DADA2/table-dada2.qza \
  --o-denoising-stats ./DADA2/stats-dada2.qza

In [ ]:
#making the directory for the merged files 
mkdir Merged

In [ ]:
cd Merged

In [ ]:
#merging the table
qiime feature-table merge \
  --i-tables ../DADA2/table-dada2.qza \
  --o-merged-table merged-table.qza

In [ ]:
#merging the representative sequences 
qiime feature-table merge-seqs \
  --i-data ../DADA2/rep-seqs-dada2.qza \
  --o-merged-data merged-rep-seqs.qza

In [ ]:
#summarization for 3
qiime feature-table summarize \
  --i-table merged-table.qza \
  --o-visualization merged-table.qzv \
  --m-sample-metadata-file sample-metadata.tsv

In [ ]:
qiime tools view merged-table.qzv

In [ ]:
qiime feature-table tabulate-seqs \
	--i-data merged-rep-seqs.qza \
	--o-visualization merged-rep-seqs.qzv

In [ ]:
#Vsearch Step 
qiime vsearch cluster-features-de-novo \
  --i-sequences ./merged-rep-seqs.qza \
  --i-table ./merged-table.qza \
  --p-perc-identity 1 \
  --o-clustered-table clustered-table.qza \
  --o-clustered-sequences clustered-seqs.qza

In [ ]:
qiime feature-table summarize \
  --i-table clustered-table.qza \
  --o-visualization clustered-table.qzv \
  --m-sample-metadata-file sample-metadata.tsv

In [ ]:
qiime tools view clustered-table.qzv

qiime feature-table tabulate-seqs \
	--i-data clustered-seqs.qza \
	--o-visualization clustered-rep-seqs.qzv

In [ ]:
conda deactivate

In [ ]:
#had to download the new qiime2 for the classification step (2024.2)
conda activate qiime2-amplicon-2024.2

In [ ]:
#Assigning taxonmy for the merged data
qiime feature-classifier classify-sklearn \
>   --i-classifier classifier_GSR-DB_V3-V4.qza \
>   --i-reads merged-rep-seqs.qza \
>   --o-classification taxonomy.qza


In [ ]:
#Assigning taxonmy with the merged and clustered data, did not do this step but the step above instead
qiime feature-classifier classify-sklearn \
  --i-classifier classifier_GSR-DB_V3-V4.qza \
  --i-reads clustered-rep-seqs.qza \
  --o-classification taxonomy1.qza

In [ ]:
qiime metadata tabulate \
>   --m-input-file taxonomy.qza \
>   --o-visualization taxonomy.qzv


In [ ]:
qiime tools view taxonomy.qzv

In [ ]:
qiime tools export \
	--input-path merged-rep-seqs.qza \
	--output-path table

In [ ]:
qiime tools export \
	--input-path taxonomy.qza \
	--output-path taxonomy

In [ ]:
qiime tools export \
	--input-path merged-table.qza \
	--output-path feature

In [ ]:
#convert feature table from biom to txt 
biom  convert \
-i feature/feature-table.biom \
--to-tsv \
-o feature/feature-table.txt
